In [1]:

import numpy as np
import pandas as pd
movies = pd.read_csv('movies_metadata.csv')
creadits =pd.read_csv('credits.csv')

C:\Users\PC\AppData\Local\Temp\ipykernel_18112\3386175969.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('movies_metadata.csv')


In [2]:
movies.iloc[0]['genres']

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [3]:
movies['id'] = movies['id'].astype(str)
creadits['id'] = creadits['id'].astype(str)
movies = movies.merge(creadits, on='id')

In [4]:
movies = movies[['id','title','overview','genres','cast','crew']]

In [5]:
movies.dropna (inplace=True)

In [6]:
duplicates = movies[movies.duplicated()]

In [7]:
movies= movies.drop_duplicates()

In [8]:
import ast

def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

In [9]:
movies['genres'] = movies['genres'].apply(convert)

In [10]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [11]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [12]:
def convert_cast(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 5:
            L.append(i['name'])
        counter+=1
    return L

In [13]:
movies['cast'] = movies['cast'].apply(convert_cast)

In [14]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [15]:
def remove_space(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [16]:
movies['cast'] = movies['cast'].apply(remove_space)
movies['crew'] = movies['crew'].apply(remove_space)
movies['genres'] = movies['genres'].apply(remove_space)

In [17]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['cast'] + movies['crew']

In [18]:
new_df = movies[['id','title','tags']]

In [19]:
new_df.loc[:,'tags'] = new_df['tags'].apply(lambda x: " ".join(x))

C:\Users\PC\AppData\Local\Temp\ipykernel_18112\4185891279.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[:,'tags'] = new_df['tags'].apply(lambda x: " ".join(x))


In [20]:
new_df.loc[:,'tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\PC\AppData\Local\Temp\ipykernel_18112\1633883480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[:,'tags'] = new_df['tags'].apply(lambda x:x.lower())


In [21]:
import nltk
from nltk.stem import PorterStemmer

In [22]:
ps = PorterStemmer()

In [23]:
def stems(text):
    T = []

    for i in text.split():
        T.append(ps.stem(i))

    return " ".join(T)

In [24]:
new_df.loc[:,'tags'] = new_df['tags'].apply(stems)

C:\Users\PC\AppData\Local\Temp\ipykernel_18112\2749765654.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[:,'tags'] = new_df['tags'].apply(stems)


In [25]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [26]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [27]:
from sklearn.metrics.pairwise import cosine_similarity
similar = cosine_similarity(vector)

MemoryError: Unable to allocate 14.7 GiB for an array with shape (44482, 44482) and data type float64

In [ ]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similar[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend("The Silence of the Lambs")